In [0]:
!pip install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/32/93/66826e2f50cefecbb0a44bd1e667316bf0a3c8e78cd1f0cdf52f5b2c5c6f/xgboost-2.1.3-py3-none-manylinux_2_28_x86_64.whl.metadata
  Obtaining dependency information for nvidia-nccl-cu12 from https://files.pythonhosted.org/packages/ed/1f/6482380ec8dcec4894e7503490fc536d846b0d59694acad9cf99f27d0e7d/nvidia_nccl_cu12-2.23.4-py3-none-manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/153.9 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/153.9 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/153.9 MB 2.3 MB/s eta 0:01:07
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.4/153.9 MB 4.2 MB/s eta 0:00:37
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.8/153.9 MB 6.4 MB/s eta 0:00:25
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/153.9 MB 7.8 MB/s eta 0:00:20
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/153.9 MB 9.1 MB/s eta 0:00:17
   ╸━━━━━━━━━━━━

In [0]:
!pip install optuna 

  Obtaining dependency information for optuna from https://files.pythonhosted.org/packages/e8/30/35111dae435c640694d616a611b7ff6b2482cfd977f8f572ff960a321d66/optuna-4.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for alembic>=1.5.0 from https://files.pythonhosted.org/packages/cb/06/8b505aea3d77021b18dcbd8133aa1418f1a1e37e432a465b14c46b2c0eaa/alembic-1.14.0-py3-none-any.whl.metadata
  Obtaining dependency information for colorlog from https://files.pythonhosted.org/packages/e3/51/9b208e85196941db2f0654ad0357ca6388ab3ed67efdbfc799f35d1f83aa/colorlog-6.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for sqlalchemy>=1.4.2 from https://files.pythonhosted.org/packages/b4/5f/95e0ed74093ac3c0db6acfa944d4d8ac6284ef5e1136b878a327ea1f975a/SQLAlchemy-2.0.36-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/d0/30/dc54f88dd4a2b5dc8a0279bdd7270e735851848b762ae

In [0]:
from pyspark.sql import functions as F
import xgboost

In [0]:
from  pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer,VectorAssembler, OneHotEncoder
from pyspark.sql.types import  BooleanType, DateType, DoubleType, IntegerType, StringType
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from xgboost.spark import SparkXGBClassifier


# processamento


In [0]:
dados = spark.read.table('hive_metastore.default.hotel_dataframe_1_csv')

In [0]:
dados.columns

['no_of_adults',
 'no_of_children',
 'no_of_weekend_nights',
 'no_of_week_nights',
 'type_of_meal_plan',
 'required_car_parking_space',
 'room_type_reserved',
 'lead_time',
 'arrival_year',
 'arrival_month',
 'arrival_date',
 'market_segment_type',
 'repeated_guest',
 'no_of_previous_cancellations',
 'no_of_previous_bookings_not_canceled',
 'avg_price_per_room',
 'no_of_special_requests',
 'booking_status',
 'is_duplicated',
 'duplicated_count',
 'data',
 'Trimestre',
 'feriado']

In [0]:
train, test = dados.randomSplit([0.8, 0.2], seed= 13)

In [0]:
#Remover target e atributos relacionados a data da matriz de atribuos
cols_to_use = train.columns
cols_to_use.remove('booking_status')
cols_to_use.remove('data') # Remover data (deve ser usada so para fins de validação/exploratório)
cols_to_use.remove('arrival_date')

In [0]:
# verificar a cardinalidade de cada coluna categorica
cat_cols_df = [col.name for col in train.schema if col.dataType == StringType()]

train.select(*[F.countDistinct(col).alias(col) for col in cat_cols_df]).show()





+-----------------+------------------+-------------------+--------------+---------+
|type_of_meal_plan|room_type_reserved|market_segment_type|booking_status|Trimestre|
+-----------------+------------------+-------------------+--------------+---------+
|                3|                 7|                  5|             2|        3|
+-----------------+------------------+-------------------+--------------+---------+



Como há pouca cardinalidade, vamos utilizar o one hot encoding

In [0]:
stages = []

In [0]:
label_transform  = StringIndexer(inputCol="booking_status", outputCol="label")
stages.append(label_transform)

In [0]:
# colunas categoricas do pipeline , isso é que são codificadas como StringType() e estão na lista de colunas permitidas
cat_cols_pipe = [col.name for col in train.schema if (col.dataType ==  StringType() and col.name in cols_to_use)]
ohe_cat_cols = []

for col in cat_cols_pipe:
    indexer = StringIndexer(inputCol=col , outputCol= f'{col}_idx', handleInvalid = 'keep')
    stages.append(indexer)
    one_hot = OneHotEncoder(inputCol=f'{col}_idx' , outputCol= f'{col}_ohe')
    stages.append(one_hot)
    ohe_cat_cols.append(f'{col}_ohe')

In [0]:
cat_cols_pipe

['type_of_meal_plan', 'room_type_reserved', 'market_segment_type', 'Trimestre']

In [0]:
# verficiar as colunas restantes
not_cat_cols = list(set(cols_to_use) - set(cat_cols_pipe))
train.select(not_cat_cols).printSchema()

root
 |-- lead_time: long (nullable = true)
 |-- is_duplicated: boolean (nullable = true)
 |-- repeated_guest: boolean (nullable = true)
 |-- required_car_parking_space: boolean (nullable = true)
 |-- no_of_previous_cancellations: long (nullable = true)
 |-- arrival_year: long (nullable = true)
 |-- no_of_adults: long (nullable = true)
 |-- feriado: boolean (nullable = true)
 |-- no_of_week_nights: long (nullable = true)
 |-- no_of_previous_bookings_not_canceled: long (nullable = true)
 |-- no_of_special_requests: long (nullable = true)
 |-- avg_price_per_room: double (nullable = true)
 |-- arrival_month: long (nullable = true)
 |-- no_of_children: long (nullable = true)
 |-- no_of_weekend_nights: long (nullable = true)
 |-- duplicated_count: long (nullable = true)



In [0]:
# vamos codificá-las tods como númericas
num_proces = VectorAssembler(inputCols = not_cat_cols , outputCol = 'num_features',handleInvalid = 'keep')
stages.append(num_proces)

In [0]:
# juntar tudo em um vetor de features
columns_pipe = ohe_cat_cols + ['num_features']
final_process =  num_proces = VectorAssembler(inputCols = columns_pipe , outputCol = 'features')
stages.append(final_process)

In [0]:
num_cluster = spark.sparkContext.defaultParallelism
print(f"numero de clusters xgb {num_cluster}")

#xgb = SparkXGBClassifier(num_workers= num_cluster, label_col="label", featuresCol = 'features')

numero de clusters xgb 8


In [0]:
# xgb_model = SparkXGBClassifier(num_workers= num_cluster, label_col="label", features_col  = 'features')
# stages.append(xgb_model)


# Ajuste optuna

In [0]:
import optuna
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder


In [0]:
pipeline = Pipeline(stages= stages)
pipelineModel = pipeline.fit(train)

In [0]:
train_transformed=  pipelineModel.transform(train)
test_transformed =  pipelineModel.transform(test)

In [0]:
# Para otimizar performance na busca dos hiperperâmetros, colocamos as transformações no cache 

train_transformed.cache()
train_transformed.take(1)

[Row(no_of_adults=0, no_of_children=1, no_of_weekend_nights=2, no_of_week_nights=4, type_of_meal_plan='Meal Plan 1', required_car_parking_space=None, room_type_reserved='Room_Type 2', lead_time=109, arrival_year=2018, arrival_month=4, arrival_date=1, market_segment_type='Online', repeated_guest=None, no_of_previous_cancellations=0, no_of_previous_bookings_not_canceled=0, avg_price_per_room=73.74, no_of_special_requests=1, booking_status='Not_Canceled', is_duplicated=False, duplicated_count=0, data=datetime.datetime(2018, 4, 1, 0, 0), Trimestre='terceiro_trimestre', feriado=None, label=0.0, type_of_meal_plan_idx=0.0, type_of_meal_plan_ohe=SparseVector(3, {0: 1.0}), room_type_reserved_idx=3.0, room_type_reserved_ohe=SparseVector(7, {3: 1.0}), market_segment_type_idx=0.0, market_segment_type_ohe=SparseVector(5, {0: 1.0}), Trimestre_idx=0.0, Trimestre_ohe=SparseVector(3, {0: 1.0}), num_features=DenseVector([109.0, 0.0, nan, nan, 0.0, 2018.0, 0.0, nan, 4.0, 0.0, 1.0, 73.74, 4.0, 1.0, 2.0, 0

In [0]:
# define a forma como as predições do nosso modelo serão avaliadas (area sobre a curva roc)
evaluator = BinaryClassificationEvaluator( rawPredictionCol = 'probability' , labelCol = 'label', metricName ='areaUnderROC' )

In [0]:
params_grid = ParamGridBuilder().build()

In [0]:
def objective(trial):
    param = {
        "max_depth" : trial.suggest_int("max_depth",3,10),
        "learning_rate"  : trial.suggest_float('learning_rate', 1e-3, 1e-1, log = True),
        "n_estimators"  : trial.suggest_categorical('n_estimators', [50,100,200,300]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
    }

    xgb = SparkXGBClassifier(num_workers= 8, 
                                   label_col="label", 
                                   features_col  = 'features',
                                   **param)
    

    cv = CrossValidator(estimator=xgb, estimatorParamMaps=params_grid, evaluator=evaluator, numFolds=5)

    xgb_model = cv.fit(train_transformed)

    pred = xgb_model.transform(train_transformed)

    auc = evaluator.evaluate(pred)

    return auc

In [0]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2024-12-15 22:56:57,300] A new study created in memory with name: no-name-0ce687c6-a315-4c12-84ef-bf36695daf81
2024-12-15 22:57:00,275 INFO XGBoost-PySpark: _fit Running xgboost-2.1.3 on 8 workers with
	booster params: {'colsample_bytree': 0.7083832235616523, 'device': 'cpu', 'learning_rate': 0.029370526301973506, 'max_depth': 5, 'objective': 'binary:logistic', 'subsample': 0.9856169225479905, 'lambda': 0.00627541409870002, 'alpha': 2.5864214668968347e-05, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
INFO:XGBoost-PySpark:Running xgboost-2.1.3 on 8 workers with
	booster params: {'colsample_bytree': 0.7083832235616523, 'device': 'cpu', 'learning_rate': 0.029370526301973506, 'max_depth': 5, 'objective': 'binary:logistic', 'subsample': 0.9856169225479905, 'lambda': 0.00627541409870002, 'alpha': 2.5864214668968347e-05, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round

In [0]:
# ajustar xgboost com melhores hyperparametros

best_param = study.best_params

best_xgb = SparkXGBClassifier(num_workers= 8, 
                                   label_col="label", 
                                   features_col  = 'features',
                                   **best_param)


xgb_model_best = best_xgb.fit(train_transformed)


2024-12-15 22:56:43,398 INFO XGBoost-PySpark: _fit Running xgboost-2.1.3 on 8 workers with
	booster params: {'objective': 'binary:logistic', 'colsample_bytree': 0.6435825218630616, 'device': 'cpu', 'learning_rate': 0.05107854376124971, 'max_depth': 9, 'subsample': 0.8883035950618128, 'lambda': 4.69102448253849e-08, 'alpha': 0.0002654458153963979, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 300}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
INFO:XGBoost-PySpark:Running xgboost-2.1.3 on 8 workers with
	booster params: {'objective': 'binary:logistic', 'colsample_bytree': 0.6435825218630616, 'device': 'cpu', 'learning_rate': 0.05107854376124971, 'max_depth': 9, 'subsample': 0.8883035950618128, 'lambda': 4.69102448253849e-08, 'alpha': 0.0002654458153963979, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 300}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}


In [0]:
train_pred_best = xgb_model_best.transform(train_transformed)
test_pred_best = xgb_model_best.transform(test_transformed)

auc_train_best = evaluator.evaluate(train_pred_best)
auc_test_best = evaluator.evaluate(test_pred_best)

In [0]:
xgb = SparkXGBClassifier(num_workers= 8, 
                                   label_col="label", 
                                   features_col  = 'features'
                                   )

#force_repartition=true
xgb_model = xgb.fit(train_transformed)



train_pred = xgb_model.transform(train_transformed)
test_pred= xgb_model.transform(test_transformed)

auc_train = evaluator.evaluate(train_pred)
auc_test = evaluator.evaluate(test_pred)

2024-12-15 22:55:05,710 INFO XGBoost-PySpark: _fit Running xgboost-2.1.3 on 8 workers with
	booster params: {'objective': 'binary:logistic', 'device': 'cpu', 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
INFO:XGBoost-PySpark:Running xgboost-2.1.3 on 8 workers with
	booster params: {'objective': 'binary:logistic', 'device': 'cpu', 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
2024-12-15 22:55:12,221 INFO XGBoost-PySpark: _fit Finished xgboost training!
INFO:XGBoost-PySpark:Finished xgboost training!


In [0]:
print(f'\n  auc train padrao : {auc_train:.3f} auc train optuna {auc_train_best:.3f}  \n auc test padrao {auc_test:.3f} auc test optuna {auc_test_best:.3f}')



  auc train padrao : 0.959 auc train optuna 0.982  
 auc test padrao 0.916 auc test optuna 0.918
